# GT and Motor Vehicle Theft in Cities in the United States

## GT Weekly City level data

In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Top 50 the most populous cities in the U.S.
region_code_dict = {
    "US-NY-501":"New York NY",
    "US-CA-807":"San Francisco-Oakland-San Jose CA", #including Oakland, CA and San Jose CA
    "US-CA-803":"Los Angeles CA", #including Long Beach, CA
    "US-CA-825":"San Diego CA", 
    "US-CA-807":"Sacramento-Stockton-Modesto CA",
    "US-AZ-753":"Phoenix AZ", #including Mesa, AZ
    "US-MI-505":"Detroit MI",
    "US-NC-517" : "Charlotte NC",
    "US-OH-535" : "Columbus OH",
    "US-FL-561" : "Jacksonville FL",
    "US-MD-511" : "Washington DC (Hagerstown MD)", # including Arlington VA
    "US-OK-650" : "Oklahoma City OK",
    "US-TX-765" : "El Paso TX",
    "US-OR-820" : "Portland OR",
    "US-MD-512" : "Baltimore MD",
    "US-WI-617" : "Milwaukee WI",
    "US-NM-790" : "Albuquerque-Santa Fe NM",
    "US-CA-866" : "Fresno-Visalia CA",
    "US-MO-616" : "Kansas City MO",
    "US-NE-652" : "Omaha NE",
    "US-CO-752" : "Colorado Springs-Pueblo CO",
    "US-NC-560" : "Raleigh-Durham (Fayetteville) NC",
    "US-VA-544" : "Norfolk-Portsmouth-Newport News VA", #including Virginia Beach, VA
    "US-FL-528" : "Miami-Ft. Lauderdale FL",
    "US-CA-800" : "Bakersfield CA",
    "US-OK-671" : "Tulsa OK",
    "US-KS-678" : "Wichita-Hutchinson KS"  ,
    "US-NV-839":"Las Vegas NV",
    "US-WA-819":"Seattle-Tacoma WA",
    "US-CO-751":"Denver CO", #including Aurora, CO
    "US-PA-504":"Philadelphia PA",
    "US-GA-524":"Atlanta GA",
    "US-IN-602":"Chicago IL",
    "US-NH-506":"Boston MA-Manchester NH",
    "CA-ON":"Ontario",
    "US-KY-529" : "Louisville KY",
    "US-TN-640" : "Memphis TN",
    "US-WI-613" : "Minneapolis-St. Paul MN",
    "US-TN-659" : "Nashville TN",
    "US-AZ-789" : "Tucson AZ",
    "US-TX-623" : "Dallas-Ft. Worth TX", # including Fort Worth, TX
    "US-TX-641" : "San Antonio TX",
    "US-TX-618" : "Houston TX",
    "US-TX-635" : "Austin TX",
}

In [3]:
filelist = os.listdir()

In [4]:
#get city name
for city_name in region_code_dict.values():
    #get the list of files which contain 'city_name'
    city_file_list = glob.glob(f"{city_name}_weekly_[0-9]*")
    #empty list to store temp_df
    list1 = []
    for file_name in city_file_list:
        temp_df = pd.read_csv(file_name, index_col=0)
        list1.append(temp_df)
    #concate temp_dfs and drop duplicated date columns and calculate the mean
    df1 = pd.concat(list1, axis=1)
    final_df_mean = df1.T.drop_duplicates().T.mean(axis = 1)
    #normalize data to maximum = 100 (similar as Google Trends)
    final_df_mean = round(final_df_mean/final_df_mean.max()*100, 3)
    
    #add index name and column name
    final_df_mean.index.name = 'date'
    final_df_mean.name = city_name
    #write into a total csv file
    final_df_mean.sort_index().to_csv(f'{city_name}_weekly_total.csv')



In [5]:
#combine all citie's mean values into one csv file
city_total_list = glob.glob(f"*total.csv")
list2 = []
for file_name in city_total_list:
    temp_df2 = pd.read_csv(file_name, index_col=0)
    list2.append(temp_df2)
    import pdb
    #pdb.set_trace()
df2 = pd.concat(list2, axis=1)


df2.sort_index().to_csv(f'all_cities_weekly.csv')

In [6]:
#keep the colums with less than 25% missing values
keep_columns = df2.columns[df2.isna().sum()/len(df2)*100 < 25]
df3 = df2[keep_columns]

In [7]:
#check for null and non-null counts of each cities
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 313 entries, 2017-01-01 to 2022-12-25
Data columns (total 33 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Albuquerque-Santa Fe NM             266 non-null    float64
 1   Atlanta GA                          312 non-null    float64
 2   Austin TX                           262 non-null    float64
 3   Baltimore MD                        253 non-null    float64
 4   Boston MA-Manchester NH             285 non-null    float64
 5   Charlotte NC                        276 non-null    float64
 6   Chicago IL                          313 non-null    float64
 7   Columbus OH                         244 non-null    float64
 8   Dallas-Ft. Worth TX                 312 non-null    float64
 9   Denver CO                           298 non-null    float64
 10  Detroit MI                          300 non-null    float64
 11  Houston TX                        

In [8]:
df3.interpolate(method ='linear', limit_direction ='backward', inplace=True)
df3.interpolate(method ='linear', limit_direction ='forward', inplace=True)

C:\Users\tosea\AppData\Local\Temp\ipykernel_73964\2683336654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.interpolate(method ='linear', limit_direction ='backward', inplace=True)
C:\Users\tosea\AppData\Local\Temp\ipykernel_73964\2683336654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.interpolate(method ='linear', limit_direction ='forward', inplace=True)


In [ ]:
#plus one day to align with city level data
df3.index = pd.to_datetime(df3.index) + pd.Timedelta(1, unit="day")

In [ ]:
df4 = df3.stack().reset_index()

In [ ]:
df3.stack().reset_index()

In [ ]:
df4

In [ ]:
df4.columns = ['date', 'dma', 'MVT_GT']

In [ ]:
df4 = df4.set_index(["date", "dma"])

In [ ]:
df4 = df4[df4.index > '2017-01-01']

## Crime Counts Data 
Ashby, M.P.J. Initial evidence on the relationship between the coronavirus pandemic and crime in the United States. Crime Sci 9, 6 (2020). https://doi.org/10.1186/s40163-020-00117-6

In [ ]:
crime_count_df = pd.read_csv("crime_counts (Matthew P J Ashby).csv")

In [ ]:
crime_count_df = crime_count_df[crime_count_df.category.str.contains("theft of vehicle")]

In [ ]:
crime_count_df2 = crime_count_df.iloc[:,[0,2,4]]

In [ ]:
crime_count_df2.columns = ['city', 'date', 'MVT_counts']

In [ ]:
crime_count_df2.index = crime_count_df2.date

In [ ]:
crime_count_df3 = crime_count_df2[crime_count_df2.index > '2017-01-01']

In [ ]:
dma_city_walk = {'Austin, TX':'Austin TX',
                 'Boston, MA':'Boston MA-Manchester NH', 
                 'Chicago, IL':'Chicago IL', 
                 'Dallas, TX':'Dallas-Ft. Worth TX',
                 'Los Angeles, CA':'Los Angeles CA', 
                 'Louisville, KY':'Louisville KY',
                 'Memphis, TN':'Memphis TN',
                 'Minneapolis, MN':'Minneapolis-St. Paul MN', 
                 'Nashville, TN':'Nashville TN',
                 'Philadelphia, PA':'Philadelphia PA',
                 'Phoenix, AZ':'Phoenix AZ',
                 'Washington, DC':'Washington DC (Hagerstown MD)'}

In [ ]:
crime_count_df3['dma'] = crime_count_df3.city.map(dma_city_walk)

In [ ]:
#filter out values without dma match
crime_count_df4 = crime_count_df3[~crime_count_df3.dma.isna()].iloc[:,[2,3]]

In [ ]:
crime_count_df4 = crime_count_df4.reset_index().set_index(["date", "dma"])

In [ ]:
crime_count_df4

In [ ]:
df4